In [16]:
import pandas as pd
import numpy as np
import librosa
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Masking
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

In [1]:
# Load the metadata
metadata_path = 'svarah\meta_speaker_stats.csv'
metadata = pd.read_csv(metadata_path)

NameError: name 'pd' is not defined

In [4]:
def sequence_generator(metadata, batch_size=32):
    while True:
        for _, row in metadata.iterrows():
            audio_path = row['audio_filepath']
            audio, _ = librosa.load('svarah/'+audio_path, sr=16000)  # Adjust the sampling rate if necessary
            sequence = np.expand_dims(audio, axis=-1)  # Add the time dimension
            sequence = sequence.reshape((1, sequence.shape[0], 1))  # Reshape for compatibility
            yield sequence, np.array(row['text'])

In [19]:
# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(metadata['text'])
total_words = len(tokenizer.word_index) + 1

In [20]:
# Build the model
model = Sequential()
model.add(Masking(mask_value=0.0, input_shape=(None, 1)))
model.add(LSTM(100, activation='relu'))
model.add(Dense(total_words, activation='softmax'))

In [21]:

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [35]:

def get_audio_sequence(audio_filepath, sequence_length):
    # Load audio file using librosa
    audio, _ = librosa.load('svarah/'+audio_filepath, sr=None)

    # Trim or pad the audio sequence to the desired length
    if len(audio) > sequence_length:
        audio_sequence = audio[:sequence_length]
    else:
        audio_sequence = np.pad(audio, (0, sequence_length - len(audio)), 'constant')

    # You might want to apply additional preprocessing steps here

    return audio_sequence

In [36]:
""" def sequence_generator(metadata, tokenizer, total_words, batch_size=32):
    while True:
        for _, row in metadata.iterrows():
            audio_path = row['audio_filepath']
            audio, _ = librosa.load('svarah/'+audio_path, sr=16000)  # Adjust the sampling rate if necessary
            sequence = np.expand_dims(audio, axis=-1)  # Add the time dimension
            sequence = sequence.reshape((1, sequence.shape[0], 1))  # Reshape for compatibility
            
            # Tokenize and one-hot encode the target text
            target_sequence = tokenizer.texts_to_sequences([row['text']])[0]
            target_sequence = to_categorical(target_sequence, num_classes=total_words)
            target_sequence = np.expand_dims(target_sequence, axis=0)

            yield sequence, target_sequence """
""" def sequence_generator(metadata, tokenizer, total_words, sequence_length, batch_size=32):
    while True:
        for _, row in metadata.iterrows():
            audio_path = row['audio_filepath']
            audio, _ = librosa.load('svarah/'+audio_path, sr=16000)  # Adjust the sampling rate if necessary
            sequence = np.expand_dims(audio, axis=-1)  # Add the time dimension
            sequence = sequence.reshape((1, sequence.shape[0], 1))  # Reshape for compatibility
            
            # Tokenize and pad the target text
            target_sequence = tokenizer.texts_to_sequences([row['text']])[0]
            target_sequence = pad_sequences([target_sequence], maxlen=sequence_length, padding='post')
            target_sequence = to_categorical(target_sequence, num_classes=total_words)
            target_sequence = np.expand_dims(target_sequence, axis=0)

            yield sequence, target_sequence """

def sequence_generator(metadata, tokenizer, total_words, sequence_length=16000, batch_size=32):
    while True:
        for index, row in metadata.iterrows():
            sequence = get_audio_sequence(row['audio_filepath'], sequence_length)
            target_sequence = tokenizer.texts_to_sequences([row['text']])[0]
            target_sequence = pad_sequences([target_sequence], maxlen=sequence_length, padding='post')
            target_sequence = to_categorical(target_sequence, num_classes=total_words)
            target_sequence = np.expand_dims(target_sequence, axis=0)
            
            yield sequence, target_sequence

In [37]:
# Train the model using the generator
# model.fit(sequence_generator(metadata), steps_per_epoch=len(metadata), epochs=10)
# model.fit(sequence_generator(metadata, tokenizer, total_words, sequence_length=16000), steps_per_epoch=len(metadata), epochs=10)
model.fit(sequence_generator(metadata, tokenizer, total_words, sequence_length=16000, batch_size=8), steps_per_epoch=len(metadata)//8, epochs=10)

MemoryError: Unable to allocate 525. MiB for an array with shape (16000, 8600) and data type float32

In [ ]:
def sequence_generator(metadata, tokenizer, total_words, sequence_length, batch_size):
    num_batches = len(metadata) // batch_size

    for batch_index in range(num_batches):
        batch_metadata = metadata[batch_index * batch_size : (batch_index + 1) * batch_size]

        sequences = []
        targets = []

        for _, row in batch_metadata.iterrows():
            audio_sequence = get_audio_sequence(row['audio_filepath'], sequence_length)
            text_sequence = tokenizer.texts_to_sequences([row['text']])[0]
            text_sequence = pad_sequences([text_sequence], maxlen=sequence_length, padding='post')
            target_sequence = to_categorical(text_sequence, num_classes=total_words)

            sequences.append(audio_sequence)
            targets.append(target_sequence)

        yield np.array(sequences), np.array(targets)

# Train the model using the generator
model.fit(
    sequence_generator(metadata, tokenizer, total_words, sequence_length=16000, batch_size=8),
    steps_per_epoch=num_batches,
    epochs=10
)
